In [1]:
import nemo.collections.asr as nemo_asr
import librosa
import numpy as np
import os
import torchaudio
import torchaudio.transforms as T
from nemo.collections.tts.models import HifiGanModel
import pytorch_lightning as pl
from torch.utils.data import DataLoader

[NeMo W 2024-10-15 09:06:27 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:254: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-10-15 09:06:27 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:265: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-10-15 09:06:27 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:325: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-10-15 09:06:27 nemo_loggin

# Converting Audio to MelSpectrograms

In [47]:
def audio_to_mel_torchaudio(audio_path, sr=22050, n_mels=80, n_fft=1024, hop_length=256, win_length=1024):
    waveform, sample_rate = torchaudio.load(audio_path)

    # Resample the audio if necessary
    if sample_rate != sr:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=sr)
        waveform = resampler(waveform)
    
    # Mel-spectrogram transformation
    mel_spectrogram_transform = T.MelSpectrogram(
        sample_rate=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=win_length,
        n_mels=n_mels
    )

    mel_spectrogram = mel_spectrogram_transform(waveform)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram.squeeze().numpy(), ref=np.max)
    
    return mel_spectrogram_db


In [58]:
audio_files_dir = '/home/keagan/Documents/projects/SelfVC/data/audios/'
output_dir = '/home/keagan/Documents/projects/SelfVC/data/mel_spectrograms/'
filelist_train = '/home/keagan/Documents/projects/SelfVC/data/train_filelist.txt'
filelist_val = '/home/keagan/Documents/projects/SelfVC/data/val_filelist.txt'

In [53]:
os.makedirs(output_dir, exist_ok=True)

# Open the filelists for writing
train_filelist = open(filelist_train, 'w')
val_filelist = open(filelist_val, 'w')

In [56]:
train_ratio = 0.95  # 95% for training, 5% for validation

In [57]:
# Process the audio files and generate mel-spectrograms
for root, dirs, files in os.walk(audio_files_dir):
    for file in files:
        if file.endswith('.wav'):
            audio_path = os.path.join(root, file)
            # mel = audio_to_mel_torchaudio(audio_path)
            
            # Save mel-spectrogram as .npy
            mel_filename = file.replace('.wav', '.npy')
            mel_filepath = os.path.join(output_dir, mel_filename)
            # np.save(mel_filepath, mel)
            
            # Determine whether to add the file to the training or validation set
            if np.random.rand() < train_ratio:
                train_filelist.write(f"{mel_filepath}|{audio_path}\n")
            else:
                val_filelist.write(f"{mel_filepath}|{audio_path}\n")

# Close the filelists
train_filelist.close()
val_filelist.close()

print(f"Filelist generation complete. Training filelist: {filelist_train}, Validation filelist: {filelist_val}")

ValueError: I/O operation on closed file.

In [59]:
train_filelist.close()
val_filelist.close()

# Convert Text to Json Dataset

In [10]:
import json

# Function to convert filelist to JSON format
def convert_txt_to_json(txt_filepath, json_filepath):
    # json_data = {"manifest_version": "1.0.0"}
    data = []
    # Open and read the .txt file
    with open(txt_filepath, 'r') as file:
        lines = file.readlines()
        
        # Process each line and create a dictionary for each entry
        for line in lines:
            mel_filepath, audio_filepath = line.strip().split('|')
            entry = {
                "audio_filepath": audio_filepath,
                "mel_filepath": mel_filepath
                
            }
            data.append(entry)
    
    # json_data["files"] = data
    # print(json_data)
    # Write the result to a JSON file
    with open(json_filepath, 'w') as json_file:
        json.dump(data, json_file,indent=4)

# Example usage
txt_file = "/home/keagan/Documents/projects/SelfVC/data/val_filelist.txt"  # Replace with your .txt file path
json_file = '/home/keagan/Documents/projects/SelfVC/data/val_filelist.json'  # Replace with your desired output path
convert_txt_to_json(txt_file, json_file)

print(f"Conversion complete. JSON saved to {json_file}")

Conversion complete. JSON saved to /home/keagan/Documents/projects/SelfVC/data/val_filelist.json


# Converting into proper Json Dataset Format

In [2]:
import os
import json
import random
import librosa

# Set paths to directories containing audio and mel spectrograms
mel_dir = "/home/keagan/Documents/projects/SelfVC/data/mel_spectrograms"
audio_dir = "/home/keagan/Documents/projects/SelfVC/data/audios"

# Function to calculate duration of audio
def get_audio_duration(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    return librosa.get_duration(y=y, sr=sr)

# List all mel and audio files
mel_files = sorted([f for f in os.listdir(mel_dir) if f.endswith('.npy')])
audio_files = sorted([f for f in os.listdir(audio_dir) if f.endswith('.wav')])

# Pair mel spectrograms and audio files
paired_data = []
for mel_file in mel_files:
    base_name = os.path.splitext(mel_file)[0]
    audio_file = base_name + ".wav"
    audio_path = os.path.join(audio_dir, audio_file)

    if os.path.exists(audio_path):
        mel_path = os.path.join(mel_dir, mel_file)
        duration = get_audio_duration(audio_path)
        
        entry = {
            "audio_filepath": audio_path,
            "mel_filepath": mel_path,
            "text": f"Placeholder text for {base_name}",
            "duration": duration
        }
        paired_data.append(entry)

# Shuffle and split into training and validation sets (80% train, 20% validation)
random.shuffle(paired_data)
split_idx = int(0.8 * len(paired_data))

train_data = paired_data[:split_idx]
val_data = paired_data[split_idx:]

# Write the train_manifest.json and val_manifest.json
with open('train_manifest.json', 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

with open('val_manifest.json', 'w') as val_file:
    json.dump(val_data, val_file, indent=4)

print(f"Train manifest saved with {len(train_data)} samples.")
print(f"Validation manifest saved with {len(val_data)} samples.")

Train manifest saved with 93200 samples.
Validation manifest saved with 23300 samples.


In [3]:
# import json

# # Path to your JSON manifest file
# manifest_path = "/home/keagan/Documents/projects/SelfVC/data/val_manifest.json"  # or "val_manifest.json"

# try:
#     with open(manifest_path, 'r') as f:
#         lines = f.readlines()
#         for idx, line in enumerate(lines):
#             line = line.strip()
#             if not line:
#                 print(f"Empty line at index {idx + 1}")
#             try:
#                 json.loads(line)
#             except json.JSONDecodeError as e:
#                 print(f"Error in line {idx + 1}: {e}")
#                 print(f"Line content: {line}")
#                 break
#         print("Manifest validated successfully if no errors!")
# except FileNotFoundError:
#     print(f"Manifest file not found: {manifest_path}")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")

Error in line 1: Expecting value: line 1 column 2 (char 1)
Line content: [
Manifest validated successfully if no errors!


In [5]:
import json

# Load the original array-style manifest
with open("/home/keagan/Documents/projects/SelfVC/data/train_manifest.json", 'r') as f:
    data = json.load(f)  # This reads the whole file as a JSON array

# Open the output file to write each entry as a separate line
with open("/home/keagan/Documents/projects/SelfVC/data/train_manifest_fixed.json", 'w') as outfile:
    for entry in data:
        json.dump(entry, outfile)
        outfile.write('\n')  # Write each JSON object to a new line